In [99]:
lms_url = "http://192.168.56.1:1234/v1/completions" # only for gemma
import requests
from tqdm import tqdm
import re
import json

In [ ]:
def get_q(context : str) -> str:    
    gem_sent_prompt = '''
    ## TASK
    For the give context, generate 3 questions and asnwer pairs such that the answer is in max 3 words
    DO not return anythign else apart from the qna pairs

    ## COntext
    {}

    ## FORMAT
Q1 : ...
A1 : ...

Q2 : ...
A2 : ...

Q3 : ...
A3 : ...

    ## OUTPUT

    '''.format(context)
    gem_sent_payload = { "prompt" : gem_sent_prompt}
    gem_sent_output = requests.post(lms_url, json=gem_sent_payload).json()["choices"][0]["text"]
    return gem_sent_output

In [5]:
text = open("../input/text_1.txt", encoding="utf-8")
data = text.read()
data

'**The Chronarium of Aethelgard: Echoes in Obsidian and Starlight**\n\n*** Chapter 1 ***\n\nThe rain in Veridia always tasted faintly of iron. This wasn\'t due to the mineral content of the soil – though it was unusually high – but to the constant, low-level hum emanating from the Chronarium, a structure dominating the western cliffs overlooking the city. It’s a place I’ve spent my entire life studying, and one that remains stubbornly resistant to complete understanding, even after decades of research. \n\nVeridia itself is nestled within the fractured continent of Aerilon, a land scarred by ancient geological upheavals and divided into twelve independent ‘Shard States,’ each clinging fiercely to its unique traditions and resources. The Shards – Solara (renowned for its sun-powered technology), Umbra (masters of shadow weaving and illusion), Kryos (a perpetually frozen nation reliant on glacial mining), Sylvana (deeply connected to the forests, possessing a symbiotic relationship with 

# 1. chunkc the text into lines

In [15]:
chunks = data.split("\n\n")
unwanted = ["---", "**"]
clean_chunks = [item for item in chunks if not any(elem in item for elem in unwanted)]
clean_chunks

["The rain in Veridia always tasted faintly of iron. This wasn't due to the mineral content of the soil – though it was unusually high – but to the constant, low-level hum emanating from the Chronarium, a structure dominating the western cliffs overlooking the city. It’s a place I’ve spent my entire life studying, and one that remains stubbornly resistant to complete understanding, even after decades of research. ",
 'Veridia itself is nestled within the fractured continent of Aerilon, a land scarred by ancient geological upheavals and divided into twelve independent ‘Shard States,’ each clinging fiercely to its unique traditions and resources. The Shards – Solara (renowned for its sun-powered technology), Umbra (masters of shadow weaving and illusion), Kryos (a perpetually frozen nation reliant on glacial mining), Sylvana (deeply connected to the forests, possessing a symbiotic relationship with the ancient trees), and so on – have historically avoided large-scale conflict, preferring

# prepare the trainign json

In [62]:
template :dict = {
                    "context": "",
                    "qas": [
                        {
                            "question": "",
                            "answer": "",
                            "id": ""
                        },
                        {
                            "question": "",
                            "answer": "",
                            "id": ""
                        },
                        {
                            "question": "",
                            "answer": "",
                            "id": ""
                        }
                    ]
                },
chunk_data = [{str(n+1):template[0]} for n,_ in enumerate(clean_chunks)]
for n,i in enumerate(chunk_data):
    # chunk_data[str(n+1)]["context"] = clean_chunks[n]
    chunk_data[n][str(n+1)] = {'context': clean_chunks[n],
   'qas': [{'answers': '', 'question': '', 'id': ''},
    {'answers': '', 'question': '', 'id': ''},
    {'answers': '', 'question': '', 'id': ''}]}
    
chunk_data

[{'1': {'context': "The rain in Veridia always tasted faintly of iron. This wasn't due to the mineral content of the soil – though it was unusually high – but to the constant, low-level hum emanating from the Chronarium, a structure dominating the western cliffs overlooking the city. It’s a place I’ve spent my entire life studying, and one that remains stubbornly resistant to complete understanding, even after decades of research. ",
   'qas': [{'answers': '', 'question': '', 'id': ''},
    {'answers': '', 'question': '', 'id': ''},
    {'answers': '', 'question': '', 'id': ''}]}},
 {'2': {'context': 'Veridia itself is nestled within the fractured continent of Aerilon, a land scarred by ancient geological upheavals and divided into twelve independent ‘Shard States,’ each clinging fiercely to its unique traditions and resources. The Shards – Solara (renowned for its sun-powered technology), Umbra (masters of shadow weaving and illusion), Kryos (a perpetually frozen nation reliant on gla

In [63]:
# n = 10
# print(chunk_data[n][str(n+1)]["context"])
# print(get_q(chunk_data[n][str(n+1)]["context"]))

# generate qna pairs

In [64]:
for n,i in enumerate(tqdm(chunk_data)):
    chunk_data[n][str(n+1)]["output"] = get_q(chunk_data[n][str(n+1)]["context"])
    # print(chunk_data[n][str(n+1)]["output"] )

100%|██████████| 61/61 [09:37<00:00,  9.47s/it]


# clean the trianing dataset

In [65]:
chunk_data

[{'1': {'context': "The rain in Veridia always tasted faintly of iron. This wasn't due to the mineral content of the soil – though it was unusually high – but to the constant, low-level hum emanating from the Chronarium, a structure dominating the western cliffs overlooking the city. It’s a place I’ve spent my entire life studying, and one that remains stubbornly resistant to complete understanding, even after decades of research. ",
   'qas': [{'answers': '', 'question': '', 'id': ''},
    {'answers': '', 'question': '', 'id': ''},
    {'answers': '', 'question': '', 'id': ''}],
   'output': "Q1 : What caused the rain's taste?\n    A1 : Iron content\n\n    Q2 : Where is the structure located?\n    A2 : Western cliffs\n\n    Q3 : What resists understanding?\n    A3 : Chronarium\n"}},
 {'2': {'context': 'Veridia itself is nestled within the fractured continent of Aerilon, a land scarred by ancient geological upheavals and divided into twelve independent ‘Shard States,’ each clinging fie

In [82]:
display(chunk_data[0]["1"]["output"])

re.findall("A{} : (.*?)\n".format(1),chunk_data[0]["1"]["output"])[0]

"Q1 : What caused the rain's taste?\n    A1 : Iron content\n\n    Q2 : Where is the structure located?\n    A2 : Western cliffs\n\n    Q3 : What resists understanding?\n    A3 : Chronarium\n"

'Iron content'

In [95]:
id = 0
for n,i in enumerate(tqdm(chunk_data)):
    for m,j in enumerate(chunk_data[n][str(n+1)]["qas"]):
        try:
            # print(chunk_data[n][str(n+1)]["output"])
            id +=1
            # chunk_data[n][str(n+1)]["qas"][m]["question"] = ""
            # chunk_data[n][str(n+1)]["qas"][m]["answers"] = ""
            chunk_data[n][str(n+1)]["qas"][m]["question"] = re.findall("Q{} : (.*?)\n".format(m+1),chunk_data[n][str(n+1)]["output"])[0]
            chunk_data[n][str(n+1)]["qas"][m]["answers"] = re.findall("A{} : (.*?)\n".format(m+1),chunk_data[n][str(n+1)]["output"])[0]
            chunk_data[n][str(n+1)]["qas"][m]["id"] = str(id)
        except KeyError as e:
            print(m)
        except IndexError as e:
            print(n)
chunk_data

100%|██████████| 61/61 [00:00<00:00, 61267.37it/s]


[{'1': {'context': "The rain in Veridia always tasted faintly of iron. This wasn't due to the mineral content of the soil – though it was unusually high – but to the constant, low-level hum emanating from the Chronarium, a structure dominating the western cliffs overlooking the city. It’s a place I’ve spent my entire life studying, and one that remains stubbornly resistant to complete understanding, even after decades of research. ",
   'qas': [{'answers': 'Iron content',
     'question': "What caused the rain's taste?",
     'id': '1'},
    {'answers': 'Western cliffs',
     'question': 'Where is the structure located?',
     'id': '2'},
    {'answers': 'Chronarium',
     'question': 'What resists understanding?',
     'id': '3'}],
   'output': "Q1 : What caused the rain's taste?\n    A1 : Iron content\n\n    Q2 : Where is the structure located?\n    A2 : Western cliffs\n\n    Q3 : What resists understanding?\n    A3 : Chronarium\n"}},
 {'2': {'context': 'Veridia itself is nestled wit

In [98]:
structured_data = {
    "data": [
        {
            "title": "Book",
            "paragraphs": [list(i.values())[0] for i in chunk_data]
        }
    ]
}
structured_data

{'data': [{'title': 'Book',
   'paragraphs': [{'context': "The rain in Veridia always tasted faintly of iron. This wasn't due to the mineral content of the soil – though it was unusually high – but to the constant, low-level hum emanating from the Chronarium, a structure dominating the western cliffs overlooking the city. It’s a place I’ve spent my entire life studying, and one that remains stubbornly resistant to complete understanding, even after decades of research. ",
     'qas': [{'answers': 'Iron content',
       'question': "What caused the rain's taste?",
       'id': '1'},
      {'answers': 'Western cliffs',
       'question': 'Where is the structure located?',
       'id': '2'},
      {'answers': 'Chronarium',
       'question': 'What resists understanding?',
       'id': '3'}],
     'output': "Q1 : What caused the rain's taste?\n    A1 : Iron content\n\n    Q2 : Where is the structure located?\n    A2 : Western cliffs\n\n    Q3 : What resists understanding?\n    A3 : Chronar

# save outuput

In [100]:
with open('../input/training_data_1.json', 'w') as f:
    json.dump(structured_data, f)